In [1]:
import numpy as np
import pandas as pd

data = []
valid_data = []
with open('trainingset_171130.txt', 'r') as f:
    data = f.readlines()
    cols = data[0].split('\t')
    for i in range(len(cols)):
        cols[i] = cols[i].strip('"')
    data = [row.split('\t') for row in data[1:]]
    for i in range(len(data)):
        for j in range(len(data[0])):
            data[i][j] = data[i][j].strip('"')
    data = pd.DataFrame(data, columns=cols)
    data = data[['Canonical_QSARr', 'EPA_category']]
data

,Canonical_QSARr,EPA_category
0,[O-][N+](=O)C1C=CC(Cl)=CC=1,2
1,NC1=CC=C(C=C1)[N+]([O-])=O,3
2,[O-][N+](=O)C1C=CC(O)=CC=1,2
3,[O-][N+](=O)C1C=CC(CCl)=CC=1,3
4,CNC1C=CC(=CC=1)[N+]([O-])=O,3
...,...,...
8989,CCOP(=S)OCC,2
8990,CP(SCCC)SCCC,1
8991,CC(O)COC(=O)C=C,2
8992,C[N+](C)(C)CCCl,3


In [2]:
with open('validationset.txt', 'r') as f:
    valid_data = f.readlines()
    cols = valid_data[0].split('\t')
    for i in range(len(cols)):
        cols[i] = cols[i].strip('"')
    valid_data = [row.split('\t') for row in valid_data[1:]]
    for i in range(len(valid_data)):
        for j in range(len(valid_data[0])):
            valid_data[i][j] = valid_data[i][j].strip('"')
    valid_data = pd.DataFrame(valid_data, columns=cols)
valid_data = valid_data[['SMILES_Original', 'EPA_category']]
valid_data

,SMILES_Original,EPA_category
0,CC(C)OC(=O)CCC\C=C/C[C@H]1[C@@H](O)C[C@@H](O)[...,1
1,CN1C(C(=O)NC2=CC=CC=N2)=C(NC2=CC=CC=N2)C2=C(C=...,1
2,CN1C(C(=O)NC2=CC=CC=N2)=C(OC(=O)C(C)(C)C)C2=C(...,1
3,CCCCC1=CC2=C(NC(NC(=O)OC)=N2)C=C1,1
4,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC2=CC=C(O)C=C2)N...,1
...,...,...
2890,C1=CC=CC=C1,3
2891,COP(=O)(OC)OC=C(Cl)Cl,1
2892,CCOP(=S)(OCC)OC1=NC(=NC(C)=C1)C(C)C,2
2893,CC(CCl)OP(=O)(OC(C)CCl)OC(C)CCl,3


In [9]:
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator, AllChem


data = data.loc[(data['EPA_category'] != 'NA') & (data['EPA_category'] != ' ')]
valid_data = valid_data.loc[(valid_data['EPA_category'] != ' ') & (valid_data['EPA_category'] != '')]

x_train = [Chem.MolFromSmiles(x) for x in data['Canonical_QSARr']]
x_train = [AllChem.GetMorganFingerprintAsBitVect(x,2,nBits=2048) for x in x_train]
x_train = np.array(x_train)
y_train = np.array([(int(x)-1) for x in data['EPA_category']])
print('train data done')

x_test = [Chem.MolFromSmiles(x) for x in valid_data['SMILES_Original']]
x_test = [AllChem.GetMorganFingerprintAsBitVect(x,2,nBits=2048) for x in x_test]
x_test = np.array(x_test, dtype=float)
y_test = np.array([(int(x)-1) for x in valid_data['EPA_category']])

print('test data done')

train data done
test data done


In [10]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.utils.data as Data
import d2lzh_pytorch as d2l

x_train = torch.tensor(x_train, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.long)
x_test = torch.tensor(x_test, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.long)


batch_size = 256
dataset = Data.TensorDataset(x_train, y_train)
train_iter = Data.DataLoader(dataset, batch_size, shuffle=True)
dataset = Data.TensorDataset(x_test, y_test)
test_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

print('done')

done


In [29]:
class Net(nn.Module):
    def __init__(self, n_feature, n_output):
        super(Net, self).__init__()
        self.h1 = torch.nn.Linear(n_feature, 32)
        self.h2 = torch.nn.Linear(32, 4)
        self.act = F.relu
        self.out = torch.nn.Linear(4, n_output)
    
    def forward(self, x):
        x = self.act(self.h1(x))
        x = self.act(self.h2(x))
        return self.out(x)

In [31]:
# 修改了很多次网络结构，依然过拟合
net = Net(n_feature=2048, n_output=4)

optimizer = torch.optim.SGD(net.parameters(), lr=0.25)
epochs = 200

d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, 'cpu', epochs)
print("Done")

training on  cpu
epoch 20, loss 0.7645, train acc 0.679, test acc 0.587, time 0.2 sec
epoch 40, loss 0.4359, train acc 0.831, test acc 0.603, time 0.2 sec
epoch 60, loss 0.1349, train acc 0.954, test acc 0.589, time 0.2 sec
epoch 80, loss 0.0796, train acc 0.971, test acc 0.592, time 0.2 sec
epoch 100, loss 0.0715, train acc 0.972, test acc 0.592, time 0.2 sec
epoch 120, loss 0.0664, train acc 0.973, test acc 0.596, time 0.2 sec
epoch 140, loss 0.0650, train acc 0.973, test acc 0.587, time 0.2 sec
epoch 160, loss 0.0599, train acc 0.973, test acc 0.591, time 0.2 sec
epoch 180, loss 0.0581, train acc 0.974, test acc 0.586, time 0.2 sec
epoch 200, loss 0.0562, train acc 0.975, test acc 0.591, time 0.2 sec
Done
